<span style="font-size:20px;">【开源实习】QAOA量子优势的衡量</span>

<span style="font-size:17px;">1 任务背景</span>

QAOA是常用的量子近似优化算法，它可以看作是在经典平均场算法上添加量子涨落。而为了衡量QAOA在哪些问题上能展现量子优势，可以使用《Mean-Field Approximate Optimization Algorithm》文中提出的Lyapunov指数（文中式70）进行衡量。用随机SK模型作为例子，用Lyapunov指数来衡量QAOA的量子优势，最终复现FIG.7的四幅图

<span style="font-size:17px;">2 任务需求及实现方案</span>

基于mindquantum的Hamiltonian构造哈密顿量来求解QAOA的基态能量E0。Mean-Field AOA算法的能量和Lyapunov指数只需通过矩阵运算即可得到 最终用matplotlib输出关于时间的函数E(t)即可

<span style="font-size:17px;">3 算法实现及效果</span>

下面基于mindquantum来实现该算法

In [1]:
import numpy as np
from scipy.linalg import expm, eigvals
from mindquantum.core.gates import X
import matplotlib.pyplot as plt
from mindquantum import Hamiltonian
from mindquantum.core.operators import QubitOperator
from scipy.sparse.linalg import eigsh

这是两个输出最终结果图的函数

第一个函数输出的是FIG7.3和FIG7.4,横坐标是归一化的时间s，纵坐标是Lyapunov指数，最终输出了Mean-field AOA算法的前三个Lyapunov指数

第二个函数输出的是FIG7.1和FIG7.2,横坐标是归一化的时间，纵坐标是E-E0，QAOA算法的20条蓝线输出的是QAOA算法前第2-21个能量E-E0 红线输出的是Mean-field AOA算法的能量E-E0

In [2]:
def draw_graph(lamda_first, lamda_second, lamda_third):
    """
    Show the top three lyapunov exponents over time.

    Args:
        lamda_first: The largest Lyapunov exponents.
        lamda_second: The second largest Lyapunov exponents.
        lamda_third: The third largest Lyapunov exponents.
    """
    x_label = [i / len(lamda_first) for i in range(len(lamda_first))]
    plt.figure(figsize=(8, 8))
    plt.plot(x_label, lamda_first, label="first", color="red")
    plt.plot(x_label, lamda_second, label="second", color="gray")
    plt.plot(x_label, lamda_third, label="third", color="gray")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xlabel("s")
    plt.ylabel("λ")
    plt.show()


def draw_graph1(eig_all, E_meanfield):
    """
    Show the top twenty exact eigen spectrum and the energy returned by the mean-field AOA over time.

    Args:
        eig_all: the exact eigen spectrum.
        E_meanfield: the energy returned by the mean-field AOA.
    """
    x_label = [i / eig_all.shape[0] for i in range(eig_all.shape[0])]
    plt.figure(figsize=(8, 8))
    for i in range(20):
        plt.plot(x_label, eig_all[:, i + 1], color="blue")
    plt.plot(x_label, E_meanfield, color="red", linestyle="--")
    plt.xlim(0, 1)
    plt.ylim(0, 4)
    plt.xlabel("s")
    plt.ylabel("E-E0")
    plt.show()

In [3]:
class Problem:
    def __init__(self, num_layers, couplings, driver=X):
        self.num_qubits = len(couplings) - 1
        self.num_layers = num_layers
        self.local_fields = couplings[:-1, -1]
        self.couplings = couplings[:-1, :-1]
        self.driver = driver

这个函数实现了量子涨落矩阵的计算

In [4]:
def fluctuation_matrix(problem, S, solution, beta, gamma):
    """
    Returns the Gaussian fluctuation matrix for a given point along the mean-field trajectories.

    Args：
        problem (Problem)
        S (np.ndarray), the initial vector of all spin vectors.
        solution (List), the solution vector.
        beta (List), the vector of QAOA driver parameters.
        gamma (List), the vector of QAOA problem parameters.

    Returns：
        matrix, the fluctuation_matrix L.
    """
    num_qubits = problem.num_qubits
    local_fields = problem.local_fields
    couplings = problem.couplings
    A = np.zeros((num_qubits, num_qubits), dtype=complex)
    B = np.zeros((num_qubits, num_qubits), dtype=complex)
    t_3 = np.diag(np.concatenate([np.ones(num_qubits), -np.ones(num_qubits)]))

    def n_ij_pm(idx, pm):
        return solution[idx] * S[idx, 0] + pm * 1j * S[idx, 1]

    for i in range(num_qubits):
        A[i, i] = (
            -beta * S[i, 0] / (1 + solution[i] * S[i, 2])
            - gamma
            * solution[i]
            * magnetization(S, local_fields, couplings, num_qubits)[i]
        )
        for j in range(i + 1, num_qubits):
            A[i, j] = gamma * couplings[i, j] * n_ij_pm(i, 1) * n_ij_pm(j, -1)
            B[i, j] = gamma * couplings[i, j] * n_ij_pm(i, 1) * n_ij_pm(j, 1)
    A += np.conj(A).T
    B += B.T
    top_block = np.hstack((A, B))
    bottom_block = np.hstack((B.conj().T, A.conj()))
    L = t_3 @ np.vstack((top_block, bottom_block))
    return L

这个函数利用上面的量子涨落矩阵来计算出lyapunov指数

In [5]:
def evolve_fluctuations(problem, t, beta, gamma):
    """
    Calculate the evolution of Gaussian fluctuations over time.

    Args：
        problem(Problem)
        t (float), the time-step of the considered annealing schedule.
        beta (List), the vector of QAOA driver parameters.
        gamma (List), the vector of QAOA problem parameters.

    Returns：
        The Lyapunov exponents characterizing the dynamics of the Gaussian fluctuations.
    """
    num_qubits = problem.num_qubits
    local_fields = problem.local_fields
    couplings = problem.couplings
    num_layers = problem.num_layers
    Lyapunov_exponent = np.zeros((num_layers, 2 * num_qubits))
    S = np.zeros((num_layers + 1, num_qubits, 3), dtype=np.float64)
    S[0, :, :] = np.array([[1.0, 0.0, 0.0] for _ in range(num_qubits)])
    S = evolve(S, local_fields, couplings, beta, gamma)
    solutions = mean_field_solution(S[-1, :, :])
    m = np.eye(2 * num_qubits, dtype=np.complex128)
    for k in range(len(gamma)):
        L = fluctuation_matrix(problem, S[k, :, :], solutions, beta[k], gamma[k])
        m = expm(-1j * t * L) @ m
        lyapunov_exponential_eig = eigvals(m @ m.conj().T)
        Lyapunov_exponent[k] = np.sort(np.real(np.log(lyapunov_exponential_eig) / 2))
    return Lyapunov_exponent

In [6]:
def magnetization(S, h, J, num_qubits):
    """Returns the magnetization vector belonging to the input 'S'."""
    return np.array(h) + np.array(
        [sum(J[i, j] * S[j, 2] for j in range(num_qubits)) for i in range(num_qubits)]
    )

In [7]:
def V_P(alpha):
    """Returns the rotation matrix resulting from the problem Hamiltonian."""
    return np.array(
        [
            [np.cos(alpha), -np.sin(alpha), 0],
            [np.sin(alpha), np.cos(alpha), 0],
            [0, 0, 1],
        ]
    )

In [8]:
def V_D(alpha):
    """Returns the rotation matrix resulting from the driver Hamiltonian."""
    return np.array(
        [
            [1, 0, 0],
            [0, np.cos(alpha), -np.sin(alpha)],
            [0, np.sin(alpha), np.cos(alpha)],
        ]
    )

这个函数是用来计算在演化过程中的所有量子的状态

In [9]:
def evolve(S, h, J, beta, gamma):
    """
    Returns the full history of the vector of spin vectors.

    Args：
        S (np.ndarray), the initial vector of all spin vectors.
        h (list), the vector of local magnetic fields of the problem Hamiltonian.
        J (np.ndarray), The coupling matrix of the problem Hamiltonian.
        beta (List), the vector of QAOA driver parameters.
        gamma (List), the vector of QAOA problem parameters.

    Returns：
        the full history of the vector of all spin vectors after a full mean-field AOA evolution.
    """
    for k in range(len(beta)):
        current_S = S[k, :, :]
        m = magnetization(current_S, h, J, len(h))
        next_S = []
        for i in range(len(h)):
            theta = -2 * gamma[k] * m[i]
            phi = -2 * beta[k]
            v_P = V_P(theta)
            v_D = V_D(phi)
            new_spin = np.dot(v_D, np.dot(v_P, S[k, i, :]))
            next_S.append(new_spin.tolist())
        S[k + 1, :, :] = next_S
    return S

这个函数是用来计算问题的能量

In [10]:
def expectation(S, h, J):
    """
    Args:
        S (np.ndarray), a vector of all spin vectors.
        h (list), the vector of local magnetic fields of the problem Hamiltonian.
        J (np.ndarray), The coupling matrix of the problem Hamiltonian.

    return:
        result (float), The energy expectation value corresponding to the supplied spin configuration.
    """
    S_z = np.array([S[i][2] for i in range(S.shape[0])])
    result = -(
        (np.transpose(h) + 0.5 * np.transpose(S_z) @ J[: len(S), : len(S)]) @ S_z
    )
    return result

In [11]:
def mean_field_solution(S):
    """Return the solution bitstring computed from a given vector of spin vectors"""
    S_z = [s[2] for s in S]
    return np.sign(S_z)

这个函数是用来计算问题哈密顿量

In [12]:
def problem_hamiltonian(problem):
    """Returns the problem Hamiltonian corresponding to 'problem'."""
    num_qubits = problem.num_qubits
    local_fields = problem.local_fields
    couplings = problem.couplings
    H = QubitOperator()
    for i in range(num_qubits):
        H += QubitOperator(f"Z{i}", coefficient=local_fields[i])
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            H += QubitOperator(f"Z{i} Z{j}", coefficient=couplings[i][j])
    return H

这个函数是用来计算驱动哈密顿量

In [13]:
def driver_hamiltonian(problem):
    """Returns the driver Hamiltonian corresponding to 'problem'."""
    num_qubits = problem.num_qubits
    H = QubitOperator()
    for i in range(num_qubits):
        H += QubitOperator(f"X{i}", coefficient=1)
    return H

<span style="font-size:17px;">4 输出结果</span>

J矩阵是原论文中的标记为"easy"的例子，来实现第一幅图的复现

步骤是：

1.初始化全体旋转量子的坐标为[1,0,0]

2.进行演化迭代，得到每一次迭代返回的量子状态

3.计算问题层能量(包括局域场和耦合矩阵)

4.计算驱动层的能量(只关于量子的x分量和)

5.计算总能量E(t)=-2*(γ(t)*问题层能量+β(t)*驱动层能量) 其中γ(t)是从0到0.5的线性函数  β(t)是从0.5到0的线性函数

横坐标是归一化的时间，纵坐标是E-E0，QAOA算法的20条蓝线输出的是QAOA算法前第2-21个能量E-E0，红线输出的是Mean-field AOA算法的能量E-E0，其中E0始终为QAOA的第一能量

In [14]:
def fig7_1(t=0.5):
    """Use the J in the 'easy' instance"""
    num_layers = 2000
    J = np.array(
        [
            [
                0.0,
                0.42132292,
                -0.25571582,
                -0.16267926,
                -0.77219702,
                0.06310006,
                0.03975859,
                0.14472543,
                0.52842128,
                0.2399146,
                -0.5426827,
            ],
            [
                0.42132292,
                0.0,
                -0.04705895,
                -0.28587394,
                0.50008779,
                0.19371417,
                0.43070882,
                0.21708425,
                -0.09432905,
                -0.07647162,
                0.0410539,
            ],
            [
                -0.25571582,
                -0.04705895,
                0.0,
                0.04119337,
                -0.03143317,
                -0.69452537,
                -0.20650927,
                0.03894435,
                -0.03114271,
                0.31022423,
                -0.25880392,
            ],
            [
                -0.16267926,
                -0.28587394,
                0.04119337,
                0.0,
                0.46391918,
                -0.46363437,
                -0.02663793,
                0.05954145,
                -0.35251299,
                -0.13726513,
                -0.07634969,
            ],
            [
                -0.77219702,
                0.50008779,
                -0.03143317,
                0.46391918,
                0.0,
                0.07847923,
                0.18372387,
                -0.47860755,
                -0.24819476,
                0.27132491,
                -0.22750873,
            ],
            [
                0.06310006,
                0.19371417,
                -0.69452537,
                -0.46363437,
                0.07847923,
                0.0,
                0.01450035,
                0.29893034,
                0.12659777,
                -0.20628129,
                -0.17935806,
            ],
            [
                0.03975859,
                0.43070882,
                -0.20650927,
                -0.02663793,
                0.18372387,
                0.01450035,
                0.0,
                0.51434209,
                -0.13210798,
                0.28592817,
                0.1470975,
            ],
            [
                0.14472543,
                0.21708425,
                0.03894435,
                0.05954145,
                -0.47860755,
                0.29893034,
                0.51434209,
                0.0,
                -0.250012,
                0.0366756,
                -0.59322573,
            ],
            [
                0.52842128,
                -0.09432905,
                -0.03114271,
                -0.35251299,
                -0.24819476,
                0.12659777,
                -0.13210798,
                -0.250012,
                0.0,
                0.26835453,
                0.00088238,
            ],
            [
                0.2399146,
                -0.07647162,
                0.31022423,
                -0.13726513,
                0.27132491,
                -0.20628129,
                0.28592817,
                0.0366756,
                0.26835453,
                0.0,
                -0.48081058,
            ],
            [
                -0.5426827,
                0.0410539,
                -0.25880392,
                -0.07634969,
                -0.22750873,
                -0.17935806,
                0.1470975,
                -0.59322573,
                0.00088238,
                -0.48081058,
                0.0,
            ],
        ]
    )
    problem = Problem(num_layers=num_layers, couplings=J)
    local_fields = problem.local_fields
    couplings = J
    num_qubits = problem.num_qubits
    beta = [t * (1 - i / num_layers) for i in range(num_layers)]
    gamma = [t * (i + 1) / num_layers for i in range(num_layers)]
    S = np.zeros((num_layers + 1, num_qubits, 3), dtype=np.float64)
    S[0, :, :] = np.array([[1, 0, 0] for _ in range(num_qubits)])
    S = evolve(S, local_fields, couplings, beta, gamma)
    Ep = [expectation(S[i], local_fields, J) for i in range(num_layers)]
    Ed = [-sum([S[i, j, 0] for j in range(num_qubits)]) for i in range(num_layers)]
    hamiltonian = [
        Hamiltonian(
            -2 * gamma[i] * problem_hamiltonian(problem)
            - 2 * beta[i] * driver_hamiltonian(problem)
        )
        for i in range(len(beta))
    ]
    eig_all = np.zeros((num_layers, 21))
    for i in range(len(gamma)):
        eigenvalues, _ = eigsh(hamiltonian[i].sparse_matrix, k=21, which="SA")
        E0 = eigenvalues[0]
        eig_all[i, 0] = E0
        for j in range(20):
            eig_all[i, j + 1] = eigenvalues[j + 1] - E0
    E0 = eig_all[:, 0]
    E_meanfield = [2 * gamma[i] * Ep[i] + 2 * beta[i] * Ed[i] for i in range(len(Ep))]
    draw_graph1(eig_all, E_meanfield - E0)


# t = 0.5
# fig7_1(t)
# remove the annotations to present FIG 7.1

J矩阵是原论文中的标记为"hard"的例子，来实现第一幅图的复现

步骤：

1.初始化全体旋转量子的坐标为[1,0,0]

2.进行演化迭代，得到每一次迭代返回的量子状态

3.计算问题层能量(包括局域场和耦合矩阵)

4.计算驱动层的能量(只关于量子的x分量和)

5.计算总能量E(t)=-2*(γ(t)*问题层能量+β(t)*驱动层能量) 其中γ(t)是从0到0.5的线性函数  β(t)是从0.5到0的线性函数

横坐标是归一化的时间，纵坐标是E-E0，QAOA算法的20条蓝线输出的是QAOA算法前第2-21个能量E-E0 红线输出的是Mean-field AOA算法的能量E-E0，其中E0始终为QAOA的第一能量

In [15]:
def fig7_2(t=0.5):
    """Use the J in the 'hard' instance"""
    num_layers = 5000
    J = np.array(
        [
            [
                0.0,
                0.20437418,
                0.18668785,
                -0.42714492,
                -0.49929211,
                -0.05025879,
                -0.08892105,
                -0.39034422,
                0.45183176,
                0.13354867,
                0.15657114,
            ],
            [
                0.20437418,
                0.0,
                0.47448795,
                -0.11285034,
                -0.19304397,
                -0.54407782,
                -0.10369845,
                -0.35005243,
                0.56644052,
                0.07814467,
                -0.23853172,
            ],
            [
                0.18668785,
                0.47448795,
                0.0,
                0.09938403,
                -0.66523568,
                -0.19274242,
                -0.08422971,
                0.16213894,
                -0.03594969,
                0.05643456,
                0.03460391,
            ],
            [
                -0.42714492,
                -0.11285034,
                0.09938403,
                0.0,
                -0.14651126,
                -0.57655351,
                -0.06858528,
                0.02506287,
                0.15807228,
                -0.05411843,
                0.10554402,
            ],
            [
                -0.49929211,
                -0.19304397,
                -0.66523568,
                -0.14651126,
                0.0,
                -0.42172937,
                -0.07213464,
                0.19051599,
                0.29572682,
                -0.46951685,
                -0.01001654,
            ],
            [
                -0.05025879,
                -0.54407782,
                -0.19274242,
                -0.57655351,
                -0.42172937,
                0.0,
                0.22313245,
                -0.11621411,
                -0.58635984,
                -0.24611057,
                -0.32501641,
            ],
            [
                -0.08892105,
                -0.10369845,
                -0.08422971,
                -0.06858528,
                -0.07213464,
                0.22313245,
                0.0,
                -0.3046252,
                -0.0152095,
                -0.22280432,
                -0.43076371,
            ],
            [
                -0.39034422,
                -0.35005243,
                0.16213894,
                0.02506287,
                0.19051599,
                -0.11621411,
                -0.3046252,
                0.0,
                -0.64309655,
                0.24139295,
                -0.11473274,
            ],
            [
                0.45183176,
                0.56644052,
                -0.03594969,
                0.15807228,
                0.29572682,
                -0.58635984,
                -0.0152095,
                -0.64309655,
                0.0,
                -0.09709491,
                -0.39243752,
            ],
            [
                0.13354867,
                0.07814467,
                0.05643456,
                -0.05411843,
                -0.46951685,
                -0.24611057,
                -0.22280432,
                0.24139295,
                -0.09709491,
                0.0,
                0.43852014,
            ],
            [
                0.15657114,
                -0.23853172,
                0.03460391,
                0.10554402,
                -0.01001654,
                -0.32501641,
                -0.43076371,
                -0.11473274,
                -0.39243752,
                0.43852014,
                0.0,
            ],
        ]
    )
    problem = Problem(num_layers=num_layers, couplings=J)
    local_fields = problem.local_fields
    couplings = J
    num_qubits = problem.num_qubits
    beta = [t * (1 - i / num_layers) for i in range(num_layers)]
    gamma = [t * (i + 1) / num_layers for i in range(num_layers)]
    S = np.zeros((num_layers + 1, num_qubits, 3), dtype=np.float64)
    S[0, :, :] = np.array([[1.0, 0.0, 0.0] for _ in range(num_qubits)])
    S = evolve(S, local_fields, couplings, beta, gamma)
    Ep = [expectation(S[i], local_fields, J) for i in range(num_layers)]
    Ed = [-sum([S[i, j, 0] for j in range(num_qubits)]) for i in range(num_layers)]
    hamiltonian = [
        Hamiltonian(
            -2 * gamma[i] * problem_hamiltonian(problem)
            - 2 * beta[i] * driver_hamiltonian(problem)
        )
        for i in range(len(beta))
    ]
    eig_all = np.zeros((num_layers, 21))
    for i in range(len(gamma)):
        eigenvalues, _ = eigsh(hamiltonian[i].sparse_matrix, k=21, which="SA")
        E0 = eigenvalues[0]
        eig_all[i, 0] = E0
        for j in range(20):
            eig_all[i, j + 1] = eigenvalues[j + 1] - E0
    E0 = eig_all[:, 0]
    E_meanfield = [2 * gamma[i] * Ep[i] + 2 * beta[i] * Ed[i] for i in range(len(Ep))]
    draw_graph1(eig_all, E_meanfield - E0)


# t = 0.5
# fig7_2(t)
# remove the annotations to present FIG 7.2

J矩阵是原论文中的标记为"easy"的例子(同FIG7.1)，来实现第三幅图的复现

步骤：

1.初始化全体旋转量子的坐标为[1,0,0]

2.进行演化迭代，得到每一次迭代返回的量子状态

3.用每一次迭代的量子状态计算每一次迭代的lyapunov矩阵

横坐标是归一化的时间s，纵坐标是Lyapunov指数，最终输出了Mean-field AOA算法的前三个Lyapunov指数

In [16]:
def fig7_3(t=0.5):
    """Use the J in the 'easy' instance"""
    num_layers = 2000
    J = np.array(
        [
            [
                0.0,
                0.42132292,
                -0.25571582,
                -0.16267926,
                -0.77219702,
                0.06310006,
                0.03975859,
                0.14472543,
                0.52842128,
                0.2399146,
                -0.5426827,
            ],
            [
                0.42132292,
                0.0,
                -0.04705895,
                -0.28587394,
                0.50008779,
                0.19371417,
                0.43070882,
                0.21708425,
                -0.09432905,
                -0.07647162,
                0.0410539,
            ],
            [
                -0.25571582,
                -0.04705895,
                0.0,
                0.04119337,
                -0.03143317,
                -0.69452537,
                -0.20650927,
                0.03894435,
                -0.03114271,
                0.31022423,
                -0.25880392,
            ],
            [
                -0.16267926,
                -0.28587394,
                0.04119337,
                0.0,
                0.46391918,
                -0.46363437,
                -0.02663793,
                0.05954145,
                -0.35251299,
                -0.13726513,
                -0.07634969,
            ],
            [
                -0.77219702,
                0.50008779,
                -0.03143317,
                0.46391918,
                0.0,
                0.07847923,
                0.18372387,
                -0.47860755,
                -0.24819476,
                0.27132491,
                -0.22750873,
            ],
            [
                0.06310006,
                0.19371417,
                -0.69452537,
                -0.46363437,
                0.07847923,
                0.0,
                0.01450035,
                0.29893034,
                0.12659777,
                -0.20628129,
                -0.17935806,
            ],
            [
                0.03975859,
                0.43070882,
                -0.20650927,
                -0.02663793,
                0.18372387,
                0.01450035,
                0.0,
                0.51434209,
                -0.13210798,
                0.28592817,
                0.1470975,
            ],
            [
                0.14472543,
                0.21708425,
                0.03894435,
                0.05954145,
                -0.47860755,
                0.29893034,
                0.51434209,
                0.0,
                -0.250012,
                0.0366756,
                -0.59322573,
            ],
            [
                0.52842128,
                -0.09432905,
                -0.03114271,
                -0.35251299,
                -0.24819476,
                0.12659777,
                -0.13210798,
                -0.250012,
                0.0,
                0.26835453,
                0.00088238,
            ],
            [
                0.2399146,
                -0.07647162,
                0.31022423,
                -0.13726513,
                0.27132491,
                -0.20628129,
                0.28592817,
                0.0366756,
                0.26835453,
                0.0,
                -0.48081058,
            ],
            [
                -0.5426827,
                0.0410539,
                -0.25880392,
                -0.07634969,
                -0.22750873,
                -0.17935806,
                0.1470975,
                -0.59322573,
                0.00088238,
                -0.48081058,
                0.0,
            ],
        ]
    )
    problem = Problem(num_layers=num_layers, couplings=J)
    beta = [t * (1 - i / problem.num_layers) for i in range(problem.num_layers)]
    gamma = [t * (i + 1) / problem.num_layers for i in range(problem.num_layers)]
    lamda = evolve_fluctuations(problem, t, beta, gamma)
    lamda_first = lamda[:, -1]
    lamda_second = lamda[:, -2]
    lamda_third = lamda[:, -3]
    draw_graph(lamda_first, lamda_second, lamda_third)


# t = 0.5
# fig7_3(t)
# remove the annotations to present FIG 7.3

J矩阵是原论文中的标记为"hard"的例子(同FIG7.2)，来实现第四幅图的复现

步骤：

1.初始化全体旋转量子的坐标为[1,0,0]

2.进行演化迭代，得到每一次迭代返回的量子状态

3.用每一次迭代的量子状态计算每一次迭代的lyapunov矩阵

横坐标是归一化的时间s，纵坐标是Lyapunov指数，最终输出了Mean-field AOA算法的前三个Lyapunov指数

In [17]:
def fig7_4(t=0.5):
    """Use the J in the 'hard' instance"""
    num_layers = 5000
    J = np.array(
        [
            [
                0.0,
                0.20437418,
                0.18668785,
                -0.42714492,
                -0.49929211,
                -0.05025879,
                -0.08892105,
                -0.39034422,
                0.45183176,
                0.13354867,
                0.15657114,
            ],
            [
                0.20437418,
                0.0,
                0.47448795,
                -0.11285034,
                -0.19304397,
                -0.54407782,
                -0.10369845,
                -0.35005243,
                0.56644052,
                0.07814467,
                -0.23853172,
            ],
            [
                0.18668785,
                0.47448795,
                0.0,
                0.09938403,
                -0.66523568,
                -0.19274242,
                -0.08422971,
                0.16213894,
                -0.03594969,
                0.05643456,
                0.03460391,
            ],
            [
                -0.42714492,
                -0.11285034,
                0.09938403,
                0.0,
                -0.14651126,
                -0.57655351,
                -0.06858528,
                0.02506287,
                0.15807228,
                -0.05411843,
                0.10554402,
            ],
            [
                -0.49929211,
                -0.19304397,
                -0.66523568,
                -0.14651126,
                0.0,
                -0.42172937,
                -0.07213464,
                0.19051599,
                0.29572682,
                -0.46951685,
                -0.01001654,
            ],
            [
                -0.05025879,
                -0.54407782,
                -0.19274242,
                -0.57655351,
                -0.42172937,
                0.0,
                0.22313245,
                -0.11621411,
                -0.58635984,
                -0.24611057,
                -0.32501641,
            ],
            [
                -0.08892105,
                -0.10369845,
                -0.08422971,
                -0.06858528,
                -0.07213464,
                0.22313245,
                0.0,
                -0.3046252,
                -0.0152095,
                -0.22280432,
                -0.43076371,
            ],
            [
                -0.39034422,
                -0.35005243,
                0.16213894,
                0.02506287,
                0.19051599,
                -0.11621411,
                -0.3046252,
                0.0,
                -0.64309655,
                0.24139295,
                -0.11473274,
            ],
            [
                0.45183176,
                0.56644052,
                -0.03594969,
                0.15807228,
                0.29572682,
                -0.58635984,
                -0.0152095,
                -0.64309655,
                0.0,
                -0.09709491,
                -0.39243752,
            ],
            [
                0.13354867,
                0.07814467,
                0.05643456,
                -0.05411843,
                -0.46951685,
                -0.24611057,
                -0.22280432,
                0.24139295,
                -0.09709491,
                0.0,
                0.43852014,
            ],
            [
                0.15657114,
                -0.23853172,
                0.03460391,
                0.10554402,
                -0.01001654,
                -0.32501641,
                -0.43076371,
                -0.11473274,
                -0.39243752,
                0.43852014,
                0.0,
            ],
        ]
    )
    problem = Problem(num_layers=num_layers, couplings=J)
    beta = [t * (1 - i / problem.num_layers) for i in range(problem.num_layers)]
    gamma = [t * (i + 1) / problem.num_layers for i in range(problem.num_layers)]
    lamda = evolve_fluctuations(problem, t, beta, gamma)
    lamda_first = lamda[:, -1]
    lamda_second = lamda[:, -2]
    lamda_third = lamda[:, -3]
    draw_graph(lamda_first, lamda_second, lamda_third)


# t = 0.5
# fig7_4(t)
# remove the annotations to present FIG 7.4